# BitFit:只对bias进行微调，其余的冻结

# 1.导包

In [ ]:
from datasets import load_dataset,Dataset

from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer

# 2.加载数据集

In [ ]:
ds = load_dataset("shibing624/alpaca-zh")
ds = ds["train"]
print(ds)

In [ ]:
ds[:3]

# 3.数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

In [ ]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human:" + example["instruction"],example["input"]]).strip() + "\n\nAssistant: ")
    # print(instruction)
    response = tokenizer(example["output"] + tokenizer.eos_token)
    # print( response)
    input_ids = instruction["input_ids"] + response["input_ids"]
    # print(input_ids)
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # print(attention_mask)
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # print(labels)
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
print(tokenized_ds)

In [ ]:
tokenizer.decode(tokenized_ds[0]["input_ids"])

In [ ]:
tokenizer.decode(list(filter(lambda x : x != -100, tokenized_ds[0]["labels"])))

# 4.创建模型

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)
# model

In [ ]:
sum(param.numel() for param in model.parameters())

## BitFit

In [ ]:

num_param = 0
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param += param.numel()
num_param

# 5.配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=2,
)

# 6.创建模型

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True)
)

# 7.训练

In [ ]:
trainer.train() # 全参数微调无法实现

# 8.模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",model=model, tokenizer=tokenizer, device=0)

In [ ]:
ipt = "Human: {}\n{}".format("考试有那些技巧","").strip() + "\n\nAssistant: "
pipe(ipt,max_length=256)